In [1]:
from cval import CVal

config_file = "../config.toml"
env_file = "../.env"

cval = CVal.init(
    config_file=config_file,
    env_file=env_file
)

/home/ssimon/github/cval/env/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


[06/23/24 16:17:38] INFO     All indexes already exist.                                                 ]8;id=499665;file:///home/ssimon/github/cval/src/cval.py\cval.py]8;;\:]8;id=745482;file:///home/ssimon/github/cval/src/cval.py#266\266]8;;\

                    INFO     Retrieval engine initialized.                                          ]8;id=500244;file:///home/ssimon/github/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=22819;file:///home/ssimon/github/cval/src/retrieval.py#29\29]8;;\

                    INFO     Ingestion engine initialized.                                          ]8;id=965701;file:///home/ssimon/github/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=939767;file:///home/ssimon/github/cval/src/ingestion.py#39\39]8;;\

In [2]:
from data import Dependency

dep = Dependency(
    project="piggymetrics",
    dependency_category="value-equality",
    option_name="EXPOSE",
    option_value="8080",
    option_type="PORT",
    option_file="Dockerfile",
    option_technology="Docker",
    dependent_option_name="server.port",
    dependent_option_value="8080",
    dependent_option_file="application.yml",
    dependent_option_type="PORT",
    dependent_option_technology="Spring-Boot"
)

response = cval.query(dependency=dep, index_name="tech-docs")

                    INFO     Select index: tech-docs.                                               ]8;id=557000;file:///home/ssimon/github/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=189366;file:///home/ssimon/github/cval/src/retrieval.py#46\46]8;;\

[06/23/24 16:17:41] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=989416;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=916299;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/23/24 16:17:42] INFO     PyTorch version 2.3.1 available.                                          ]8;id=749393;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/datasets/config.py\config.py]8;;\:]8;id=214706;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/datasets/config.py#58\58]8;;\

/home/ssimon/github/cval/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[06/23/24 16:17:43] INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=865563;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=508447;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

...

[06/23/24 16:17:45] INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=155618;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=23308;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

                    INFO     Len of retrieved nodes of query 2: 5                                  ]8;id=183213;file:///home/ssimon/github/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=231268;file:///home/ssimon/github/cval/src/retrieval.py#142\142]8;;\

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.89it/s]


                    INFO     Len of retrieved nodes after reranking: 3                             ]8;id=920811;file:///home/ssimon/github/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=915388;file:///home/ssimon/github/cval/src/retrieval.py#150\150]8;;\

[06/23/24 16:17:50] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=27440;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=738338;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

In [5]:
print(type(response))
print(response)

<class 'llama_index.core.base.llms.types.CompletionResponse'>
```json
{
  "plan": "1. Identify the purpose of the EXPOSE directive in the Dockerfile and the server.port configuration in the Spring-Boot application.yml. 2. Determine if the EXPOSE directive in Dockerfile and server.port in application.yml need to have the same value for the application to function correctly. 3. Check if there are any references or documentation that explicitly state that these two configuration options must be identical. 4. Evaluate if the Docker container's exposed port (EXPOSE) and the Spring-Boot server's listening port (server.port) need to match for proper communication and functionality. 5. Conclude whether a value-equality dependency exists based on the findings.",
  "rationale": "The EXPOSE directive in the Dockerfile is used to inform Docker that the container listens on the specified network ports at runtime. The server.port configuration in the Spring-Boot application.yml specifies the port on